# Visualising Emissions

It is often useful to be able to produce visualisations of profiling data, including CO~2~ estimates.

Here we will use the [Weights & Biases](https://wandb.ai) platform to construct a simple profiling dashboard for our code.

### Step 1: 
Visit https://wandb.ai/authorize

### Step 2: 
Log in with your GitHub account or with Google

### Step 3: 
Copy API key (under the profile dropdown on the right-hand side)

### Step 4: 
Log in to W&B 

- In VSCode Terminal, run python3 (or python)

- `import wandb`

- `wandb.login()`

- Enter your choice: 2

- Paste the copied API key


### Step 5: 
Exit python environment from terminal

### Step 6: 
Run your script with `wandb.init()` and then `wandb.log`, and `wandb.finish()` to end the wandb run.

In [ ]:
import numpy as np
import psutil, time, os, gc, warnings
from codecarbon import EmissionsTracker
import wandb

warnings.filterwarnings('ignore')


# Function to get memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Memory in MB

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)

# Function to get run stats with multiple iterations
def run_stats(project_name, analysis_func, num_runs):
    results = []
    emissions = []
    runtimes = []
    cpu_usages = []
    mem_usages = []
    
    # Initialize wandb run
    wandb.init(project=project_name, reinit=True)
    
    for run_idx in range(num_runs):

        gc.collect()  # Clear memory before each run
        tracker = EmissionsTracker()
        tracker.start()
        
        start_mem = get_memory_usage()
        start_cpu = get_cpu_usage()
        start_time = time.time()
        
        result = analysis_func()
        
        end_time = time.time()
        end_cpu = get_cpu_usage()
        end_mem = get_memory_usage()
       
        emission = tracker.stop()

        results.append(result)

        emissions.append(emission)
        cpu_usages.append((start_cpu + end_cpu) / 2)  # Average CPU usage during run
        mem_usages.append(end_mem - start_mem)
        runtimes.append(end_time - start_time)

        
        # Log per-run metrics to wandb
        wandb.log({
            "Runtime (s)": runtimes[-1],
            "CPU Usage (%)": cpu_usages[-1],
            "RAM Usage (M)": mem_usages[-1],
            "Emissions (kgCO2)": emission,
        })

        # Print result
        print(result)

    # End wandb run
    wandb.finish()  
    
    # Compute statistics
    metrics = {
        "mean runtime (s)": np.mean(runtimes),
        "std runtime (s)": np.std(runtimes) if num_runs > 1 else 0,
        "mean RAM (M)": np.mean(mem_usages),
        "std RAM (M)": np.std(mem_usages) if num_runs > 1 else 0,
        "mean CPU (%)": np.mean(cpu_usages),
        "std CPU (%)": np.std(cpu_usages) if num_runs > 1 else 0,
        "mean emissions (kgCO2)": np.mean(emissions),
        "std emissions (kgCO2)": np.std(emissions) if num_runs > 1 else 0,
    }
    
    return (results, metrics)

In [ ]:
def fib(n):
    if(n == 0):
        return 0
    if(n == 1):
        return 1
    return fib(n - 1) + fib(n - 2)

def fibs(n):
    return [fib(i) for i in range(n)]

def fib_task():
    return fibs(35)

(results, metrics) = run_stats("fib", fib_task, 10)

print(results[0])

for k in metrics.keys():
    print(f"{k}: {metrics[k]}")

## Task: C0~2~ visualisation

Try making a W&B dashboard for your pathfinding code.

---